In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import datetime
from src import supertrend, VWAP2, smc, wavetrend3d, atr, pivotPoints
from src.backtesting_lib import evaluatePairs
import pandas_ta as ta
from scipy.signal import argrelextrema
import csv

ModuleNotFoundError: No module named 'src'

In [ ]:
# New try without obs and local minima and maxima yet

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import warnings
import numpy as np

warnings.filterwarnings("ignore")

class WL(Strategy):
    atr_window = 13
    tradehourStart = 0
    order = 10
    longs = False
    shorts = True
    tradehourEnd = 23
    candlecountEntry = 10
    def init(self):
        self.vwap = self.I(VWAP2, self.data.df, 0)
        self.vwapu1 = self.I(VWAP2, self.data.df, 1)
        self.ema = self.I(ta.ema, self.data.df["Close"], length=100)
        self.vwapl1 = self.I(VWAP2, self.data.df, 2)
        self.vwapu2 = self.I(VWAP2, self.data.df, 3)
        self.lastdiffs = []
        self.vwapl2 = self.I(VWAP2, self.data.df, 4)
        self.vwapu3 = self.I(VWAP2, self.data.df, 5)
        self.vwapl3 = self.I(VWAP2, self.data.df, 6)
        self.atr = self.I(atr, self.data.df, self.atr_window)
        self.trade_opened = False
        self.sholong = 0
        self.pivotLower = self.I(pivotPoints, self.data.df, self.order, 0)
        self.pivotUpper = self.I(pivotPoints, self.data.df, self.order, 1)
        self.tagged = False
        self.taggeddev = 0
        self.idk = self.data.df
        self.wavetrendMirror = self.I(wavetrend3d, self.idk, 5,False,"30T", True)
        self.wavetrend = self.I(wavetrend3d, self.idk, 5,False ,"30T", False)
        self.stoploss = 0
        self.candlecount = 0
        self.prevvalueU = 0
        self.prevvalueB = 0
        self.taggedcounter = 0
        self.hochpunkte = []
        self.tiefpunkte = []
        self.supertrendu = self.I(supertrend, self.data.df, 10, 2.5, 0)
        self.supertrendl = self.I(supertrend, self.data.df, 10, 2.5, 1)
        

    def next(self):
        if(self.pivotUpper[-1] > 0 and self.pivotUpper[-1] != self.prevvalueU):
            lower_val = 100000
            higher_val = 0
            for i in range(1, 3):
                if(self.data.Low[-i] < lower_val):
                    lower_val = self.data.Low[-i]
                if(self.data.High[-i] > higher_val):
                    higher_val = self.data.High[-i]
            self.hochpunkte.append([higher_val, higher_val, self.candlecount])
            self.prevvalueU = self.pivotUpper[-1]
        if(self.pivotLower[-1] > 0 and self.pivotLower[-1] != self.prevvalueB):
            lower_val = 100000
            higher_val = 0
            for i in range(1, 3):
                if(self.data.Low[-i] < lower_val):
                    lower_val = self.data.Low[-i]
                if(self.data.High[-i] > higher_val):
                    higher_val = self.data.High[-i]
            self.tiefpunkte.append([higher_val, lower_val, self.candlecount])
            self.prevvalueB = self.pivotLower[-1]

        
        if not self.trade_opened and self.data.index[-1].hour >= self.tradehourStart and self.data.index[-1].hour <= self.tradehourEnd:
            if((self.data.High[-1] > self.vwapu2[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)) or (self.data.High[-1] > self.vwapu3[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu3[-1] + (self.atr[-1]/2)) and self.shorts):
                if self.wavetrendMirror[-1] > self.wavetrend[-1]:
                    self.tagged = True
                    if(self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)):
                        self.taggeddev = 2
                    else:
                        self.taggeddev = 3
                    self.sholong = 1
            elif((self.data.Low[-1] < self.vwapl2[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)) or (self.data.Low[-1] < self.vwapl3[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl3[-1] - (self.atr[-1]/2)) and self.longs):
                if self.wavetrendMirror[-1] < self.wavetrend[-1]:
                    self.tagged = True
                    if(self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)):
                        self.taggeddev = 2
                    else:
                        self.taggeddev = 3
                    self.sholong = 0
            elif (self.tagged == True and self.sholong == 1):
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.High[-1] > self.vwapu2[-1] + (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.High[-1] > self.vwapu3[-1] + (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                if self.taggedcounter < self.candlecountEntry:
                    if ha_open > (ha_high - ((ha_high - ha_low)*0.12) and self.data.Close[-1]>self.vwapu1[-1]):
                        self.trade_opened = True
                        self.stoploss = self.data.Close[-1] + (self.atr[-1]/2)
                        def berechnen(kapital, stoploss, entry):
                            diff = abs(stoploss - entry)
                            if(diff < 0.000001):
                                diff = 0
                            self.lastdiffs.append(diff)
                            if(diff == 0):
                                size = 0
                            else: 
                                size = abs((kapital/10000) / abs(diff))
                            return int(round(size,0))
                        sizeO = berechnen(self.equity, self.stoploss, self.data.Close[-1])
                        self.sell(size=sizeO)
                    self.taggedcounter += 1
                else: 
                    self.tagged = False
                    self.taggedcounter = 0                    
            elif (self.tagged == True and self.sholong == 0):
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.Low[-1] < self.vwapl2[-1] - (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.Low[-1] < self.vwapl3[-1] - (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                else:
                    if self.taggedcounter < self.candlecountEntry:
                        if ha_open < (ha_low + (ha_high - ha_low) * 0.12) and self.data.Close[-1]<self.vwapl1[-1]:
                            self.stoploss = self.data.Close[-1] - (self.atr[-1]/2)
                            self.trade_opened = True
                            def berechnen(kapital, stoploss, entry):
                                diff = abs(stoploss - entry)
                                if(diff < 0.000001):
                                    diff = 0
                                self.lastdiffs.append(diff)
                                if(diff == 0):
                                    size = 0
                                else: 
                                    size = abs((kapital/10000) / abs(diff))
                                return int(round(size,0))
                            sizeO = berechnen(self.equity, self.stoploss, self.data.Close[-1])
                            self.buy(size=sizeO)
                        else: 
                            self.tagged = False
                            self.taggedcounter = 0
                        self.taggedcounter += 1
        elif self.trade_opened:
            if self.sholong == 0:
                if(self.supertrendl[-2] > 0 and self.supertrendu[-1] > 0) or self.data.Close[-1] < self.stoploss:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False
            elif self.sholong == 1:
                if(self.supertrendu[-2] > 0 and self.supertrendl[-1] > 0) or self.data.Close[-1] > self.stoploss:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False


In [ ]:
# New try without obs and local minima and maxima yet

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import warnings
import numpy as np

warnings.filterwarnings("ignore")

class MeanReverse(Strategy):
    atr_window = 13
    tradehourStart = 0
    order = 10
    longs = True
    bigon = True
    wavetrendOn = True
    wavetrendTiemframe = "30T"
    wavetrendCock = True
    cockSize = 12
    shorts = True
    tradehourEnd = 23
    candlecountEntry = 10
    def init(self):
        self.supertrenduBIG = self.I(supertrend, self.data.df, 200, 50, 0)
        self.supertrendlBIG = self.I(supertrend, self.data.df, 200, 50, 1)
        self.vwap = self.I(VWAP2, self.data.df, 0)
        self.vwapu1 = self.I(VWAP2, self.data.df, 1)
        self.vwapl1 = self.I(VWAP2, self.data.df, 2)
        self.vwapu2 = self.I(VWAP2, self.data.df, 3)
        self.lastdiffs = []
        self.vwapl2 = self.I(VWAP2, self.data.df, 4)
        self.vwapu3 = self.I(VWAP2, self.data.df, 5)
        self.vwapl3 = self.I(VWAP2, self.data.df, 6)
        self.atr = self.I(atr, self.data.df, self.atr_window)
        self.trade_opened = False
        self.sholong = 0
        self.tagged = False
        self.taggeddev = 0
        self.idk = self.data.df
        self.stoploss = 0
        self.candlecount = 0
        self.idk = self.data.df
        self.wavetrend = self.I(wavetrend3d, self.idk, self.cockSize, self.wavetrendCock, self.wavetrendTiemframe, False)
        self.wavetrendMirror = self.I(wavetrend3d, self.idk, self.cockSize, self.wavetrendCock, self.wavetrendTiemframe, True)
        self.prevvalueU = 0
        self.prevvalueB = 0
        self.taggedcounter = 0
        self.hochpunkte = []
        self.tiefpunkte = []
        self.supertrendu = self.I(supertrend, self.data.df, 10, 2.5, 0)
        self.supertrendl = self.I(supertrend, self.data.df, 10, 2.5, 1)
        

    def next(self):
        if not self.trade_opened and self.data.index[-1].hour >= self.tradehourStart and self.data.index[-1].hour <= self.tradehourEnd:
            if((self.data.High[-1] > self.vwapu2[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)) or (self.data.High[-1] > self.vwapu3[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu3[-1] + (self.atr[-1]/2))) and self.shorts:
                if self.supertrenduBIG[-1] > 0.0001 or self.bigon == False:
                    if self.wavetrendMirror[-1] > self.wavetrend[-1] or self.wavetrendOn == False:
                        self.tagged = True
                        if(self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)):
                            self.taggeddev = 2
                        else:
                            self.taggeddev = 3
                        self.sholong = 1
            elif((self.data.Low[-1] < self.vwapl2[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)) or (self.data.Low[-1] < self.vwapl3[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl3[-1] - (self.atr[-1]/2))) and self.longs:
                if self.supertrendlBIG[-1] > 0.0001 or self.bigon == False:
                    if self.wavetrendMirror[-1] < self.wavetrend[-1] or self.wavetrendOn == False:
                        self.tagged = True
                        if(self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)):
                            self.taggeddev = 2
                        else:
                            self.taggeddev = 3
                        self.sholong = 0
            elif (self.tagged == True and self.sholong == 1):
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.High[-1] > self.vwapu2[-1] + (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.High[-1] > self.vwapu3[-1] + (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                if self.taggedcounter < self.candlecountEntry:
                    if ha_open > (ha_high - ((ha_high - ha_low)*0.10) and self.data.Close[-1]>self.vwapu1[-1]):
                        self.trade_opened = True
                        self.stoploss = self.data.Close[-1] + (self.atr[-1]/4)
                        def berechnen(kapital, stoploss, entry):
                            diff = abs(stoploss - entry)
                            if(diff < 0.0000000001):
                                diff = 0
                            self.lastdiffs.append(diff)
                            if(diff == 0):
                                size = 0
                            else: 
                                size = abs((kapital/10000) / abs(diff))
                            return int(round(size,0))
                        sizeO = berechnen(self.equity, self.stoploss, self.data.Close[-1])
                        self.buy(size=sizeO)
                    self.taggedcounter += 1
                else: 
                    self.tagged = False
                    self.taggedcounter = 0                    
            elif (self.tagged == True and self.sholong == 0):
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.Low[-1] < self.vwapl2[-1] - (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.Low[-1] < self.vwapl3[-1] - (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                else:
                    if self.taggedcounter < self.candlecountEntry:
                        if ha_open < (ha_low + (ha_high - ha_low) * 0.10) and self.data.Close[-1]<self.vwapl1[-1]:
                            self.stoploss = self.data.Close[-1] - (self.atr[-1]/4)
                            self.trade_opened = True
                            def berechnen(kapital, stoploss, entry):
                                diff = abs(stoploss - entry)
                                if(diff < 0.00000000001):
                                    diff = 0
                                self.lastdiffs.append(diff)
                                if(diff == 0):
                                    size = 0
                                else: 
                                    size = abs((kapital/10000) / abs(diff))
                                return int(round(size,0))
                            sizeO = berechnen(self.equity, self.stoploss, self.data.Close[-1])
                            self.sell(size=sizeO)
                        else: 
                            self.tagged = False
                            self.taggedcounter = 0
                        self.taggedcounter += 1
        elif self.trade_opened:
            if self.sholong == 0:
                if(self.supertrendl[-2] > 0 and self.supertrendu[-1] > 0) or self.data.Close[-1] < self.stoploss or self.data.Close[-1] > self.vwapu2[-1]:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False
            elif self.sholong == 1:
                if(self.supertrendu[-2] > 0 and self.supertrendl[-1] > 0) or self.data.Close[-1] > self.stoploss or self.data.Close[-1] < self.vwapl2[-1]:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False


# __Was dieser Algo zurzeit macht:__ 
---
    Er überprüft erstmal ob wir in der richtigen Timezone sind um einen Trade zu eröffnen (tradehourStart, tradehourEnd). 
    Danach geht er durch jeden einzigen Hoch und Tiefpunkt, welcher in den letzten deletePivotPoints da ist und guckt, ob sich der die jetzige letzte Candle Close in der Pivot Point Range befindet und in der zweiten oder dritten Deviation range (mit atr) befindet und removed den jetzigen Pivot Point, da der bereits getagged wurde. Das macht er mit Hoch und Tiefpunkten. Danach berechnet er im nächsten Path die heikin ashis und schaut, ob die letzten 5 candles vor dieser candle unter dem pivot point sind. Dabei meine ich das einfach so, dass.
    Dann schaut der Algo, ob ein Heikin Ashi Entry signal da ist und ob der unser jetziger Value sich noch über/unter der ersten Deviation befindet, damit man nciht zu spät entered. Außerdem entered er nur einen trade wenn der nach weniger oder gleich candledcountEntry Candles passiert ist. 
    Dann geht er jeweils einen short oder long ein und geht raus, wenn der Stoploss welcher aus den High oder Low der letzten 5 candles beim entry entsteht oder wenn der Supertrend einen Trendwechsel indicated.

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import warnings
import numpy as np

warnings.filterwarnings("ignore")



class PIES(Strategy):

    #Hier die Werte die ihr abändern könnt
    shorts = True
    longs = False
    candlecountEntry = 10 # Nach wie vielen noch geschaut wird ob buy/sell signal kommt
    stoplossCandles = 1 # Beim Entry -> Wie viele candles er nach hinten schauen soll für sl
    tradehourStart = 6 #UTC Time
    tradehourEnd = 22 #UTC Time
    wavetrendOn = False
    atr_window = 13
    stoplossratio = 0.5
    timeframeOBs = "6T"  #T for Minute and H for Hour
    timeframeWavetrend = "6T"
    cockOn = False
    cockSize = 6
    delete = 1000
    orderblocklength = 10
    feesOn = True

    def init(self):
        # Store orderblock values
        self.orderblocksU = []
        self.orderblocksB = []
        self.startKap = self.equity
        # If orderblock got hit and price is above/below upperbands/lowerbands
        self.tagged = False
        self.taggedprice = 0
        self.taggeddev = 0
        self.lastdiffs = []
        self.length = len(self.data.df)
        # Counter of how many candles it was ago when orderblock got tagged
        self.taggedcounter = 0
        # Overall candle count
        self.candlecount = 17
        self.trade_opened = False
        self.index = 0
        self.fees = 0
        self.log = []
        self.kapitalverfolgung = 1000000
        self.stoploss = 0
        self.sholong = 0  #0 stands for long trade and 1 for short trade
        # Values that act as a buffer for the orderblocks after they got hit
        self.orderblocklow = 0
        self.orderblockhigh = 0 
        self.done = True
        self.lastKapital = self.equity
        self.idk = self.data.df
        self.enter = 0
        # Supertrend Indicator
        self.supertrendu = self.I(supertrend, self.data.df, 10, 2.5, 0)
        self.supertrendl = self.I(supertrend, self.data.df, 10, 2.5, 1)
        # VWAP Indicator
        self.upperOB = self.I(smc, self.data.df, length=self.orderblocklength, band=1, timeframe=self.timeframeOBs)
        self.lowerOB = self.I(smc, self.data.df, length=self.orderblocklength, band=0, timeframe=self.timeframeOBs)
        self.wavetrendMirror = self.I(wavetrend3d, self.idk, self.cockSize,self.cockOn ,self.timeframeWavetrend, True)
        self.wavetrend = self.I(wavetrend3d, self.idk, self.cockSize,self.cockOn ,self.timeframeWavetrend, False)
        self.vwap = self.I(VWAP2, self.data.df, 0)
        self.vwapu1 = self.I(VWAP2, self.data.df, 1)
        self.vwapl1 = self.I(VWAP2, self.data.df, 2)
        self.vwapu2 = self.I(VWAP2, self.data.df, 3)
        self.vwapl2 = self.I(VWAP2, self.data.df, 4)
        self.vwapu3 = self.I(VWAP2, self.data.df, 5)
        self.vwapl3 = self.I(VWAP2, self.data.df, 6)
        self.prevvalueU = 0
        self.prevvalueB = 0
        # ATR Indicator
        self.atr = self.I(atr, self.data.df, self.atr_window)

    def next(self):
        # Add orderblocks
        # Nimmt orderblöcke raus die älter als 1000 candles sind
        if(len(self.orderblocksB) > 0):
            if(self.candlecount - self.orderblocksB[0][2] > self.delete and len(self.orderblocksB) > 1):
                self.orderblocksB = self.orderblocksB[1:]
        if(len(self.orderblocksU) > 0):
            if(self.candlecount - self.orderblocksU[0][2] > self.delete and len(self.orderblocksU) > 1):
                self.orderblocksU = self.orderblocksU[1:]

        # Added die Orderblöcke zur Liste
        if(self.upperOB[-1] > 0 and self.upperOB[-1] != self.prevvalueU):
            self.orderblocksU.append([self.data.High[-1], self.data.Low[-1], self.candlecount])
            self.prevvalueU = self.upperOB[-1]
        if(self.lowerOB[-1] > 0 and self.lowerOB[-1] != self.prevvalueB):
            self.orderblocksB.append([self.data.High[-1], self.data.Low[-1], self.candlecount])
            self.prevvalueB = self.lowerOB[-1]

        # If no trade is open
        if not self.trade_opened and self.data.index[-1].hour >= self.tradehourStart and self.data.index[-1].hour <= self.tradehourEnd:
            # If it didnt hit orderblock + upper/lower band yet
            if self.tagged==False:
                # For loops to iterate through the orderblocks and check if they got hit
                if self.shorts:
                    if self.wavetrendMirror[-1] > self.wavetrend[-1] or self.wavetrendOn == False:
                        for high, low, candle in self.orderblocksU:
                            if self.data.High[-1] > low and self.data.High[-1] < high and self.candlecount - candle < 1000 and ((self.data.High[-1] > self.vwapu2[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)) or (self.data.High[-1] > self.vwapu3[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu3[-1] + (self.atr[-1]/2))):
                                self.tagged = True
                                if(self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)):
                                    self.taggeddev = 2
                                else:
                                    self.taggeddev = 3
                                self.taggedprice = self.data.High[-1]
                                self.orderblocklow = low
                                self.sholong = 1
                                self.orderblocksU.remove([high, low, candle])
                                break
                if self.longs:
                    if self.wavetrendMirror[-1] < self.wavetrend[-1] or self.wavetrendOn == False:
                        for high, low, candle in self.orderblocksB:
                            if self.data.Low[-1] < high and self.data.Low[-1] > low and self.candlecount - candle < 1000 and ((self.data.Low[-1] < self.vwapl2[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)) or (self.data.Low[-1] < self.vwapl3[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl3[-1] - (self.atr[-1]/2))):
                                self.tagged = True
                                if(self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)):
                                    self.taggeddev = 2
                                else:
                                    self.taggeddev = 3
                                self.orderblockhigh = high
                                self.sholong = 0
                                self.taggedprice = self.data.Low[-1]
                                self.orderblocksB.remove([high, low, candle])
                                break

            # If it hit and we have to go short
            elif (self.tagged == True and self.sholong == 1):
                # Heikin Ashi Candles)
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.High[-1] > self.vwapu2[-1] + (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.High[-1] > self.vwapu3[-1] + (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                else:
                    if self.taggedcounter < self.candlecountEntry:
                        if self.data.High[-1] < self.orderblocklow and ha_open > (ha_high - ((ha_high - ha_low)*0.12) and self.data.Close[-1]>self.vwapu1[-1]):
                            self.trade_opened = True
                            self.stoploss = self.data.High[-2]
                            
                            def berechnen(kapital, stoploss, entry):
                                    diff = abs(stoploss - entry)
                                    if(diff < 0.0000001):
                                        diff = abs(sum(self.lastdiffs) / len(self.lastdiffs))
                                    self.lastdiffs.append(diff)
                                    size = abs((kapital/10000) / abs(diff))
                                    return int(round(size,0))
                            sizeO = berechnen(self.equity, self.data.High[-2], self.data.Close[-1])
                            try:
                                if(self.data.Close[-1]*sizeO < self.kapitalverfolgung):
                                    self.kapitalverfolgung = self.kapitalverfolgung - self.data.Close[-1]*sizeO
                                    self.index += 1
                                    self.enter = self.data.Close[-1]
                                    self.done = False
                                self.sell(size=sizeO)
                                if self.feesOn:
                                    self.fees += sizeO * self.data.Close[-1] * 3 / 100000
                            except:
                                print("Something went wrong")
                        self.taggedcounter += 1
                    else: 
                        self.tagged = False
                        self.taggedcounter = 0
            # If it hit and we have to go long
            elif (self.tagged == True and self.sholong == 0):
                # Heikin Ashi Candles
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.Low[-1] < self.vwapl2[-1] - (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.Low[-1] < self.vwapl3[-1] - (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                else:
                    if self.taggedcounter < self.candlecountEntry:
                        if self.data.Low[-1] > self.orderblockhigh and ha_open < (ha_low + (ha_high - ha_low) * 0.12) and self.data.Close[-1]<self.vwapl1[-1]:
                            self.stoploss = self.data.Low[-2] # Set stoploss to high value
                            self.trade_opened = True
                            def berechnen(kapital, stoploss, entry):
                                diff = abs(stoploss - entry)
                                if(diff < 0.0000000001):
                                    diff = abs(sum(self.lastdiffs) / len(self.lastdiffs))
                                self.lastdiffs.append(diff)
                                size = abs((kapital/10000) / abs(diff))
                                return int(round(size,0))
                            sizeO = berechnen(self.equity, self.data.High[-2], self.data.Close[-1])
                            try:
                                if(self.data.Close[-1]*sizeO < self.kapitalverfolgung):
                                    self.kapitalverfolgung = self.kapitalverfolgung - self.data.Close[-1]*sizeO
                                    self.index += 1
                                    self.enter = self.data.Close[-1]
                                    self.done = False
                                self.buy(size=sizeO)
                                if self.feesOn:
                                    self.fees += sizeO * self.data.Close[-1] * 3 / 100000
                            except:
                                print("Something went wrong")
                            self.trade_opened = True
                        self.taggedcounter += 1
                    else: 
                        self.tagged = False
                        self.taggedcounter = 0
        
        elif self.trade_opened: # Stop loss path
            if self.sholong == 0:
                if(self.data.Low[-1] < self.stoploss and not self.done):
                    temp = self.stoploss * abs(self.position.size)
                    fees = temp - (temp * 3/100000)
                    self.kapitalverfolgung = self.kapitalverfolgung + fees
                    self.done = True
                    self.log.append([self.data.index[-1], self.index, self.kapitalverfolgung, self.position.size, self.enter, self.stoploss, "Long", self.kapitalverfolgung - self.lastKapital])
                    self.lastKapital = self.kapitalverfolgung
                elif(self.supertrendl[-2] > 0 and self.supertrendu[-1] > 0 and not self.done):
                    temp = self.data.Close[-1] * abs(self.position.size)
                    fees = temp - (temp * 3/100000)
                    self.kapitalverfolgung = self.kapitalverfolgung + fees
                    self.done = True
                    self.log.append([self.data.index[-1], self.index, self.kapitalverfolgung, self.position.size, self.enter, self.stoploss, "Long", self.kapitalverfolgung - self.lastKapital])   
                    self.lastKapital = self.kapitalverfolgung
                if(self.supertrendl[-2] > 0 and self.supertrendu[-1] > 0) or self.data.Close[-1] < self.stoploss:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False
            elif self.sholong == 1:
                if(self.data.High[-1] > self.stoploss and not self.done):
                    temp = (self.enter - (self.stoploss - self.enter)) * abs(self.position.size)
                    fees = temp - (temp * 3/100000)
                    self.kapitalverfolgung = self.kapitalverfolgung + fees
                    self.done = True
                    self.log.append([self.data.index[-1], self.index, self.kapitalverfolgung, self.position.size, self.enter, self.stoploss, "Short", self.kapitalverfolgung - self.lastKapital])
                    self.lastKapital = self.kapitalverfolgung
                elif(self.supertrendu[-2] > 0 and self.supertrendl[-1] > 0 and not self.done):
                    temp = (self.enter + (self.enter - self.data.Close[-1])) * abs(self.position.size)
                    fees = temp - (temp * 3/100000)
                    self.kapitalverfolgung = self.kapitalverfolgung + fees
                    self.done = True
                    self.log.append([self.data.index[-1], self.index, self.kapitalverfolgung, self.position.size, self.enter, self.stoploss, "Short", self.kapitalverfolgung - self.lastKapital])
                    self.lastKapital = self.kapitalverfolgung
                if(self.supertrendu[-2] > 0 and self.supertrendl[-1] > 0) or self.data.Close[-1] > self.stoploss:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False
        self.candlecount = self.candlecount + 1
        if(len(self.data.Close) == self.length):
            print(f"Die Fees betragen: {self.fees}")
            headers = ['Time', 'Index', 'Equity', 'Shares', 'Entry', 'Stoploss', 'Short or Long', 'Profit']
            def save_list_as_csv(data_list, file_path):
                with open(file_path, "w", newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow(headers)
                    for item in data_list:
                        formatted_row = [
                            str(item[0]),  # Convert the timestamp to string format
                            item[1],
                            "{:.4f}".format(item[2]), 
                            item[3],
                            "{:.5f}".format(item[4]),
                            "{:.5f}".format(item[5]),
                            item[6],
                            "{:.4f}".format(item[7])
                        ]
                        writer.writerow(formatted_row)
            folder_path = "./results_lists/"
            file_name = "list.csv"
            file_path = folder_path + file_name
            save_list_as_csv(self.log, file_path)

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import warnings
import numpy as np

warnings.filterwarnings("ignore")
    
class WL(Strategy):
    atr_window = 13
    tradehourStart = 6
    order = 15
    longs = True
    shorts = True
    stoplosscandles = 5
    tradehourEnd = 20
    deletePivotPoints = 1000
    candlecountEntry = 8
    volumeRange = 10
    def init(self):
        self.vwap = self.I(VWAP2, self.data.df, 0)
        self.vwapu1 = self.I(VWAP2, self.data.df, 1)
        self.vwapl1 = self.I(VWAP2, self.data.df, 2)
        self.vwapu2 = self.I(VWAP2, self.data.df, 3)
        self.lastdiffs = []
        self.sma = self.I(ta.vwma, self.data.df["Close"], self.data.df["Volume"], self.volumeRange)
        self.vwapl2 = self.I(VWAP2, self.data.df, 4)
        self.vwapu3 = self.I(VWAP2, self.data.df, 5)
        self.vwapl3 = self.I(VWAP2, self.data.df, 6)
        self.atr = self.I(atr, self.data.df, self.atr_window)
        self.pivotLower = self.I(pivotPoints, self.data.df, self.order, 0)
        self.pivotUpper = self.I(pivotPoints, self.data.df, self.order, 1)
        self.supertrendu = self.I(supertrend, self.data.df, 10, 2.5, 0)
        self.supertrendl = self.I(supertrend, self.data.df, 10, 2.5, 1)
        self.sholong = 0
        self.supertrendFiltero = self.I(supertrend, self.data.df, 200, 50, 0)
        self.supertrendFilteru = self.I(supertrend, self.data.df, 200, 50, 1)
        self.trade_opened = False
        self.tagged = False
        self.taggeddev = 0
        self.stoploss = 0
        self.candlecount = 0
        self.prevvalueU = 0
        self.prevvalueB = 0
        self.saveHochpunkt = 0
        self.saveTiefpunkt = 0
        self.taggedcounter = 0
        self.hochpunkte = []
        self.tiefpunkte = []
        

    def next(self):
        # ---- Hoch und Tiefpunkte entnehmen die älter als 1k candles sind----
        if(len(self.tiefpunkte) > 0):
            if(self.candlecount - self.tiefpunkte[0][2] > self.deletePivotPoints and len(self.tiefpunkte) > 1):
                self.tiefpunkte = self.tiefpunkte[1:]
        if(len(self.hochpunkte) > 0):
            if(self.candlecount - self.hochpunkte[0][2] > self.deletePivotPoints and len(self.hochpunkte) > 1):
                self.hochpunkte = self.hochpunkte[1:]
        # ---- Füge Hoch und Tiefpunkte zu Liste hinzu hinzu ----
        if(self.pivotUpper[-1] > 0 and self.pivotUpper[-1] != self.prevvalueU):
            self.hochpunkte.append([self.data.High[-1], self.data.Low[-1], self.candlecount])
            self.prevvalueU = self.pivotUpper[-1]
        if(self.pivotLower[-1] > 0 and self.pivotLower[-1] != self.prevvalueB):
            self.tiefpunkte.append([self.data.High[-1], self.data.Low[-1], self.candlecount])
            self.prevvalueB = self.pivotLower[-1]
        # ---- Tagged Path beginnt hier ----        
        if not self.trade_opened and self.data.index[-1].hour >= self.tradehourStart and self.data.index[-1].hour <= self.tradehourEnd:
            if self.shorts and self.data.Close[-1] < self.sma[-1]:
                for high, low, candle in self.hochpunkte:
                    if((self.data.High[-1] > low and self.data.High[-1] < high and self.candlecount - candle > self.order and self.data.High[-1] > self.vwapu2[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)) 
                        or (self.data.High[-1] > low and self.data.High[-1] < high and self.candlecount - candle > self.order and self.data.High[-1] > self.vwapu3[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu3[-1] + (self.atr[-1]/2))) and self.shorts:
                        self.tagged = True
                        if(self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)):
                            self.taggeddev = 2
                        else:
                            self.taggeddev = 3
                        self.sholong = 1
                        self.saveHochpunkt = high
                        self.hochpunkte.remove([high, low, candle])
                        break
            if self.longs and self.data.Close[-1] > self.sma[-1]:
                for high, low, candle in self.tiefpunkte:
                    if((self.data.Low[-1] > low and self.data.Low[-1] < high and self.candlecount - candle > self.order and self.data.Low[-1] < self.vwapl2[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)) or 
                        (self.data.Low[-1] > low and self.data.Low[-1] < high and self.candlecount - candle > self.order and self.data.Low[-1] < self.vwapl3[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl3[-1] - (self.atr[-1]/2))) and self.longs:
                        self.tagged = True
                        if(self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)):
                            self.taggeddev = 2
                        else:
                            self.taggeddev = 3
                        self.sholong = 0
                        self.saveTiefpunkt = low
                        self.tiefpunkte.remove([high, low, candle])
                        break
            # ---- Tagged Path endet hier ----
            # ---- Heikin Ashi Signal beginnt hier für Shorts----
            if (self.tagged == True and self.sholong == 1):
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.High[-1] > self.vwapu2[-1] + (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.High[-1] > self.vwapu3[-1] + (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                if self.taggedcounter < self.candlecountEntry:
                    if ha_open > (ha_high - ((ha_high - ha_low)*0.10) and self.data.Close[-1]>self.vwapu1[-1]):
                        self.trade_opened = True
                        self.stoploss = 0
                        for i in range(1, self.stoplosscandles):
                            if self.data.High[-i] > self.stoploss:
                                self.stoploss = self.data.High[-i]
                        def berechnen(kapital, stoploss, entry):
                            diff = abs(stoploss - entry)
                            if(diff < 0.000001):
                                diff = 0
                            self.lastdiffs.append(diff)
                            if(diff == 0):
                                size = 1
                            else: 
                                size = abs((kapital/10000) / abs(diff))
                            return int(round(size,0))
                        sizeO = berechnen(self.equity, self.stoploss, self.data.Close[-1])
                        self.sell(size=sizeO)
                    self.taggedcounter += 1
                else: 
                    self.tagged = False
                    self.taggedcounter = 0   
            # ---- Heikin Ashi Signal beginnt hier für Longs ----                 
            elif (self.tagged == True and self.sholong == 0):
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.Low[-1] < self.vwapl2[-1] - (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.Low[-1] < self.vwapl3[-1] - (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                else:
                    if self.taggedcounter < self.candlecountEntry:
                        if ha_open < (ha_low + (ha_high - ha_low) * 0.10) and self.data.Close[-1]<self.vwapl1[-1]:
                            self.stoploss = 1000000
                            for i in range(1, self.stoplosscandles):
                                if self.data.Low[-i] < self.stoploss:
                                    self.stoploss = self.data.Low[-i]
                            self.trade_opened = True
                            def berechnen(kapital, stoploss, entry):
                                diff = abs(stoploss - entry)
                                if(diff < 0.000001):
                                    diff = 0
                                self.lastdiffs.append(diff)
                                if(diff == 0):
                                    size = 1
                                else: 
                                    size = abs((kapital/10000) / abs(diff))
                                return int(round(size,0))
                            sizeO = berechnen(self.equity, self.stoploss, self.data.Close[-1])
                            self.buy(size=sizeO)
                        self.taggedcounter += 1
                    else: 
                        self.tagged = False
                        self.taggedcounter = 0
        # ---- Stoploss/Take Profit Path ----
        elif self.trade_opened:
            if self.sholong == 0:
                if(self.supertrendl[-2] > 0 and self.supertrendu[-1] > 0) or self.data.Close[-1] < self.stoploss:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False
            elif self.sholong == 1:
                if(self.supertrendu[-2] > 0 and self.supertrendl[-1] > 0) or self.data.Close[-1] > self.stoploss:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False
        self.candlecount += 1